In [1]:
# !pip install 'zarr<3'
# !pip install timm
# !pip install openslide-python tiffslide
# !pip install cupy-cuda12x  # for CUDA 12.x

In [2]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


## WSI Syn

In [3]:
# Cell 1: Load model and create predictor with synthetic MIF generation
import torch
from vitaminp import VitaminPDual  # ← Changed from VitaminPSyn
from vitaminp.inference import WSIPredictor

# Load your VitaminPDual model (trained on real MIF data)
model = VitaminPDual(model_size='base').to('cuda')

# Load your model checkpoint
checkpoint = torch.load('checkpoints/vitamin_p_dual_base_fold2_best.pth', map_location='cuda')
model.load_state_dict(checkpoint)
model.eval()

# Create predictor with synthetic MIF generation enabled
predictor = WSIPredictor(
    model=model,
    device='cuda',
    gan_checkpoint_path="checkpoints_gan/pix2pix_he_to_mif_best.pth",
    target_mpp=0.50,
    magnification=20,
    use_synthetic_mif=True  # ← NEW: Enable synthetic MIF generation
)
print("✅ Predictor initialized with synthetic MIF generation")

# Cell 2: Run inference - just provide H&E slide!
# The GAN will automatically generate synthetic MIF in the background
results = predictor.predict(
    wsi_path='test_images/region_5000x5000_standard.tif',  # ← Only H&E needed!
    output_dir='results_syn_nuclei',
    branch='he_nuclei',  # ← Use he_nuclei (will use MIF predictions internally)
    filter_tissue=True,
    tissue_threshold=0.05,
    clean_overlaps=True,
    save_geojson=True,
    detection_threshold=0.5,
    min_area_um=5.0,
)
print(f"✅ Found {results['num_detections']} nuclei in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Building H&E encoder with DINOv2-base
Building MIF encoder with DINOv2-base
Building shared encoder with DINOv2-base
✓ VitaminPDual initialized with base backbone
  Embed dim: 768 | Decoder dims: [768, 384, 192, 96]


2026-01-22 22:08:41 - WSIPredictor - INFO - 🎨 Loading GAN generator from checkpoints_gan/pix2pix_he_to_mif_best.pth
2026-01-22 22:08:41 - WSIPredictor - INFO -    ⚠ No architecture metadata in checkpoint, using provided params
2026-01-22 22:08:41 - WSIPredictor - INFO -    Architecture: attention=True, spectral_norm=False, residual_blocks=4
2026-01-22 22:08:42 - WSIPredictor - INFO -    ✓ GAN generator loaded successfully
2026-01-22 22:08:42 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-22 22:08:42 - WSIPredictor - INFO -   Device: cuda
2026-01-22 22:08:42 - WSIPredictor - INFO -   Model type: VitaminPDual (dual-modality with synthetic MIF generation)
2026-01-22 22:08:42 - WSIPredictor - INFO -   Patch size: 512
2026-01-22 22:08:42 - WSIPredictor - INFO -   Overlap: 64
2026-01-22 22:08:42 - WSIPredictor - INFO -   Magnification: 20
2026-01-22 22:08:42 - WSIPredictor - INFO -   Synthetic MIF: Enabled
2026-01-22 22:08:42 - WSIPredictor - INFO - 🔄 Using MIF predictions for he_

✅ Predictor initialized with synthetic MIF generation
   Virtual upscaled size: 9505x9505 (from 5000x5000)
   Scanning 22x22 tile grid...


2026-01-22 22:09:29 - WSIPredictor - INFO -    ✓ Created 484 tiles (22x22 grid)
2026-01-22 22:09:29 - WSIPredictor - INFO -    ✓ Tissue tiles: 484/484 (100.0%)
2026-01-22 22:09:29 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on he_nuclei...


   Tissue dilation: 471 → 484 tiles (+13 boundary tiles)


Processing tiles: 100%|██████████| 484/484 [01:35<00:00,  5.09it/s]
2026-01-22 22:11:04 - WSIPredictor - INFO -    ✓ Extracted 15361 instances from tiles (before cleaning)
2026-01-22 22:11:04 - WSIPredictor - INFO -    🔍 DEBUG: Tile configuration:
2026-01-22 22:11:04 - WSIPredictor - INFO -       - Tile size: 512px
2026-01-22 22:11:04 - WSIPredictor - INFO -       - Overlap: 64px
2026-01-22 22:11:04 - WSIPredictor - INFO -       - Grid: 22x22 tiles
2026-01-22 22:11:04 - WSIPredictor - INFO -    🔍 DEBUG: Cells near tile boundaries (within 64px): 6242
2026-01-22 22:11:04 - WSIPredictor - INFO -    🔍 DEBUG: Potential overlap rate: 40.6%
2026-01-22 22:11:04 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-22 22:11:04 - WSIPredictor - INFO -    🔍 DEBUG: Total cells before cleaning: 15361
2026-01-22 22:11:04 - WSIPredictor - INFO -    🔍 DEBUG: Edge cells found: 7464
2026-01-22 22:11:04 - WSIPredictor - INFO -    🔍 DEBUG: Will clean ALL instances (not jus

✅ Found 11446 nuclei in 155.42s
   Output saved to: results_syn_nuclei


In [3]:
# Cell 1: Load model and create predictor with improved GAN
from vitaminp import VitaminPSyn
from vitaminp.inference import WSIPredictor

# Load your VitaminPSyn model
model = VitaminPSyn(model_size='base').to('cuda')
# Load your model checkpoint here if needed
# checkpoint = torch.load('path/to/model_checkpoint.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Create predictor with IMPROVED GAN checkpoint
predictor = WSIPredictor(
    model=model,
    device='cuda',
    gan_checkpoint_path="checkpoints_gan/pix2pix_he_to_mif_best.pth"
    # ✓ Will auto-detect architecture from checkpoint if available
)

print("✅ Predictor initialized with improved GAN model")


# Cell 2: Run inference - just provide H&E slide!
# The improved GAN will automatically generate synthetic MIF in the background
results = predictor.predict(
    wsi_path='test_images/CRC31-he_chunk_14.png',  # ← Only H&E needed!
    output_dir='results_syn_nuclei',
    branch='syn_nuclei',  # ← Automatically uses improved GAN + MIF predictions
    filter_tissue=True,
    tissue_threshold=0.05,
    clean_overlaps=True,
    save_geojson=True,
    detection_threshold=0.5,
    min_area_um=5.0,
)

print(f"✅ Found {results['num_detections']} nuclei in {results['processing_time']:.2f}s")
print(f"   Output saved to: {results['output_dir']}")

Building H&E encoder with DINOv2-base
Building Synthetic MIF encoder with DINOv2-base
Building shared encoder with DINOv2-base
✓ VitaminPSyn initialized with base backbone
  Embed dim: 768 | Decoder dims: [768, 384, 192, 96]


2026-01-22 21:00:16 - WSIPredictor - INFO - 🎨 Loading GAN generator from checkpoints_gan/pix2pix_he_to_mif_best.pth
2026-01-22 21:00:16 - WSIPredictor - INFO -    ⚠ No architecture metadata in checkpoint, using provided params
2026-01-22 21:00:16 - WSIPredictor - INFO -    Architecture: attention=True, spectral_norm=False, residual_blocks=4
2026-01-22 21:00:17 - WSIPredictor - INFO -    ✓ GAN generator loaded successfully
2026-01-22 21:00:17 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-22 21:00:17 - WSIPredictor - INFO -   Device: cuda
2026-01-22 21:00:17 - WSIPredictor - INFO -   Model type: VitaminPSyn (H&E + Synthetic MIF)
2026-01-22 21:00:17 - WSIPredictor - INFO -   Patch size: 512
2026-01-22 21:00:17 - WSIPredictor - INFO -   Overlap: 64
2026-01-22 21:00:17 - WSIPredictor - INFO -   Magnification: 40
2026-01-22 21:00:17 - WSIPredictor - INFO -    ⚠ No MPP in metadata, using default: 0.2500 μm/px
2026-01-22 21:00:17 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-

✅ Predictor initialized with improved GAN model
   Virtual upscaled size: 486x486 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  1.48it/s]
2026-01-22 21:00:18 - WSIPredictor - INFO -    ✓ Extracted 0 instances from tiles (before cleaning)
2026-01-22 21:00:18 - WSIPredictor - INFO - 💾 Saving results to results_syn_nuclei...
2026-01-22 21:00:18 - WSIPredictor - INFO -    Loading full image for visualization...
2026-01-22 21:00:18 - WSIPredictor - INFO - ✅ Complete! 0 detections in 0.79s


✅ Found 0 nuclei in 0.79s
   Output saved to: results_syn_nuclei


In [11]:
# ========================================================================
# EXAMPLE 2: VitaminPSyn - Synthetic Cell Segmentation
# ========================================================================
print("\n" + "="*80)
print("EXAMPLE 2: VitaminPSyn - Synthetic Cell Detection")
print("="*80)

results = predictor.predict(
    wsi_path='test_images/ovarian-he_chunk_92.png',
    output_dir='results_syn_cell',
    branch='syn_cell',  # ← Uses synthetic MIF for cell segmentation
    filter_tissue=True,
    clean_overlaps=True,
    save_geojson=True,
    detection_threshold=0.5,
    min_area_um=20.0,  # Larger threshold for cells
)

print(f"✅ Found {results['num_detections']} cells in {results['processing_time']:.2f}s")


2026-01-19 19:00:08 - WSIPredictor - INFO -    ⚠ No MPP in metadata, using default: 0.2630 μm/px
2026-01-19 19:00:08 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-19 19:00:08 - WSIPredictor - INFO -    WSI MPP: 0.2630 μm/px
2026-01-19 19:00:08 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-19 19:00:08 - WSIPredictor - INFO -    Scale factor: 1.00x
2026-01-19 19:00:08 - WSIPredictor - INFO -    Min area filter: 20.0 μm² = 289 pixels²
2026-01-19 19:00:08 - WSIPredictor - INFO - 🎨 Opening H&E WSI for synthetic MIF generation: test_images/ovarian-he_chunk_92.png
2026-01-19 19:00:08 - WSIPredictor - INFO -    ✓ Size: 512x512 pixels
2026-01-19 19:00:08 - WSIPredictor - INFO - 📐 Extracting tile positions...
2026-01-19 19:00:08 - WSIPredictor - INFO -    ✓ Created 1 tiles (1x1 grid)
2026-01-19 19:00:08 - WSIPredictor - INFO -    ✓ Tissue tiles: 1/1 (100.0%)
2026-01-19 19:00:08 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on syn_cell...



EXAMPLE 2: VitaminPSyn - Synthetic Cell Detection
   Virtual upscaled size: 512x512 (from 512x512)
   Scanning 1x1 tile grid...
   Tissue dilation: 1 → 1 tiles (+0 boundary tiles)


Processing tiles: 100%|██████████| 1/1 [00:00<00:00,  5.37it/s]
2026-01-19 19:00:08 - WSIPredictor - INFO -    ✓ Extracted 90 instances from tiles (before cleaning)
2026-01-19 19:00:08 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-19 19:00:08 - WSIPredictor - INFO -    Found 36 edge cells to check for overlaps
2026-01-19 19:00:08 - WSIPredictor - INFO - Starting overlap cleaning with 90 detections
2026-01-19 19:00:08 - WSIPredictor - INFO - Split detections: 0 mid, 90 edge
2026-01-19 19:00:08 - WSIPredictor - INFO - Iteration 1: Found 0 overlaps, keeping 90/90 detections
2026-01-19 19:00:08 - WSIPredictor - INFO - No more overlaps found, stopping early
2026-01-19 19:00:08 - WSIPredictor - INFO - Cleaning complete: 90 -> 90 (0 removed)
2026-01-19 19:00:08 - WSIPredictor - INFO -    ✓ After cleaning: 90 instances
2026-01-19 19:00:08 - WSIPredictor - INFO - 💾 Saving results to results_syn_cell...
2026-01-19 19:00:08 - WSIPredictor - INFO -    Loadi

✅ Found 90 cells in 0.35s


In [14]:
# Create predictor with GAN checkpoint
predictor = WSIPredictor(
    model=model,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.5013,
    magnification=20,
    gan_checkpoint_path="checkpoints/pix2pix_he_to_mif_final.pth"  # ← GAN for synthetic MIF
)
# ========================================================================
# EXAMPLE 3: Process Multiple Branches at Once
# ========================================================================
print("\n" + "="*80)
print("EXAMPLE 3: Process Multiple Syn Branches")
print("="*80)

results_multi = predictor.predict(
    wsi_path='test_images/MS004_HE_center_5000x5000.png',
    output_dir='results_syn_multi',
    branches=['syn_nuclei', 'syn_cell'],  # ← Both branches!
    filter_tissue=True,
    clean_overlaps=True,
    save_geojson=True,
)

for branch_name, branch_results in results_multi.items():
    print(f"\n{branch_name}:")
    print(f"  Detections: {branch_results['num_detections']}")
    print(f"  Time: {branch_results['processing_time']:.2f}s")
    print(f"  Output: {branch_results['output_dir']}")



2026-01-19 19:07:04 - WSIPredictor - INFO - 🎨 Loading GAN generator from checkpoints/pix2pix_he_to_mif_final.pth
2026-01-19 19:07:04 - WSIPredictor - INFO -    ✓ GAN generator loaded successfully
2026-01-19 19:07:04 - WSIPredictor - INFO - WSIPredictor initialized:
2026-01-19 19:07:04 - WSIPredictor - INFO -   Device: cuda
2026-01-19 19:07:04 - WSIPredictor - INFO -   Model type: VitaminPSyn (H&E + Synthetic MIF)
2026-01-19 19:07:04 - WSIPredictor - INFO -   Patch size: 512
2026-01-19 19:07:04 - WSIPredictor - INFO -   Overlap: 64
2026-01-19 19:07:04 - WSIPredictor - INFO -   Magnification: 20
2026-01-19 19:07:04 - WSIPredictor - INFO - 
2026-01-19 19:07:04 - WSIPredictor - INFO - Processing branch: syn_nuclei
2026-01-19 19:07:04 - WSIPredictor - INFO - ============================================================



EXAMPLE 3: Process Multiple Syn Branches


2026-01-19 19:07:05 - WSIPredictor - INFO -    ⚠ No MPP in metadata, using default: 0.5013 μm/px
2026-01-19 19:07:05 - WSIPredictor - INFO - 🔍 Resolution matching:
2026-01-19 19:07:05 - WSIPredictor - INFO -    WSI MPP: 0.5013 μm/px
2026-01-19 19:07:05 - WSIPredictor - INFO -    Model training MPP: 0.2630 μm/px
2026-01-19 19:07:05 - WSIPredictor - INFO -    Scale factor: 1.91x
2026-01-19 19:07:05 - WSIPredictor - INFO -    Min area filter: 3.0 μm² = 12 pixels²
2026-01-19 19:07:05 - WSIPredictor - INFO - 🎨 Opening H&E WSI for synthetic MIF generation: test_images/MS004_HE_center_5000x5000.png
2026-01-19 19:07:06 - WSIPredictor - INFO -    ✓ Size: 5000x5000 pixels
2026-01-19 19:07:06 - WSIPredictor - INFO - 📐 Extracting tile positions...


   Virtual upscaled size: 9530x9530 (from 5000x5000)
   Scanning 22x22 tile grid...


2026-01-19 19:07:08 - WSIPredictor - INFO -    ✓ Created 484 tiles (22x22 grid)
2026-01-19 19:07:08 - WSIPredictor - INFO -    ✓ Tissue tiles: 484/484 (100.0%)
2026-01-19 19:07:08 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on syn_nuclei...


...

   Virtual upscaled size: 9530x9530 (from 5000x5000)
   Scanning 22x22 tile grid...


2026-01-19 19:09:07 - WSIPredictor - INFO -    ✓ Created 484 tiles (22x22 grid)
2026-01-19 19:09:07 - WSIPredictor - INFO -    ✓ Tissue tiles: 484/484 (100.0%)
2026-01-19 19:09:07 - WSIPredictor - INFO - 🧠 Running predictions and extracting instances on syn_cell...


   Tissue dilation: 467 → 484 tiles (+17 boundary tiles)


Processing tiles: 100%|██████████| 484/484 [01:25<00:00,  5.66it/s]
2026-01-19 19:10:32 - WSIPredictor - INFO -    ✓ Extracted 57314 instances from tiles (before cleaning)
2026-01-19 19:10:33 - WSIPredictor - INFO - 🧹 Cleaning overlapping instances at tile boundaries...
2026-01-19 19:10:33 - WSIPredictor - INFO -    Found 17255 edge cells to check for overlaps
2026-01-19 19:10:36 - WSIPredictor - INFO - Starting overlap cleaning with 57314 detections
2026-01-19 19:10:36 - WSIPredictor - INFO - Split detections: 0 mid, 57314 edge
2026-01-19 19:10:45 - WSIPredictor - INFO - Iteration 1: Found 15669 overlaps, keeping 41645/57314 detections
2026-01-19 19:10:52 - WSIPredictor - INFO - Iteration 2: Found 1074 overlaps, keeping 40571/41645 detections
2026-01-19 19:10:58 - WSIPredictor - INFO - Iteration 3: Found 127 overlaps, keeping 40444/40571 detections
2026-01-19 19:11:04 - WSIPredictor - INFO - Iteration 4: Found 22 overlaps, keeping 40422/40444 detections
2026-01-19 19:11:11 - WSIPredic


syn_nuclei:
  Detections: 36792
  Time: 118.10s
  Output: results_syn_multi/syn_nuclei

syn_cell:
  Detections: 40409
  Time: 167.60s
  Output: results_syn_multi/syn_cell


In [ ]:
# ========================================================================
# EXAMPLE 4: VitaminPFlex - MIF Only (for comparison)
# ========================================================================
print("\n" + "="*80)
print("EXAMPLE 4: VitaminPFlex - Real MIF (comparison)")
print("="*80)

from vitaminp import VitaminPFlex

# Setup Flex model
model_flex = VitaminPFlex(model_size='large').to(device)
model_flex.load_state_dict(torch.load("checkpoints/vitamin_p_flex_large_fold2_best.pth", map_location=device))
model_flex.eval()

# Create channel config
config = ChannelConfig(
    nuclear_channel=0,
    membrane_channel=[1, 2],
    membrane_combination='max',
    channel_names={0: 'SYTO13', 1: 'Cy3', 2: 'TexasRed'}
)

# Create predictor (no GAN needed for Flex)
predictor_flex = WSIPredictor(
    model=model_flex,
    device='cuda',
    patch_size=512,
    overlap=64,
    target_mpp=0.263,
    magnification=20,
    mif_channel_config=config
)

# Run MIF inference
results_mif = predictor_flex.predict(
    wsi_path='test_images/ovarian-he_chunk_92.png',
    output_dir='results_flex_mif',
    branch='mif_nuclei',
    filter_tissue=False,
    clean_overlaps=True,
    save_geojson=True,
    detection_threshold=0.5,
    min_area_um=5.0,
)

print(f"✅ Found {results_mif['num_detections']} nuclei in {results_mif['processing_time']:.2f}s")

# ========================================================================
# COMPARISON: Syn vs Real MIF
# ========================================================================
print("\n" + "="*80)
print("COMPARISON SUMMARY")
print("="*80)
print(f"Synthetic MIF (from H&E): {results['num_detections']} nuclei")
print(f"Real MIF:                 {results_mif['num_detections']} nuclei")
print("="*80)